In [2]:
# Mount Google Drive for data access (essential for Colab persistence)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Running in Google Colab - Drive mounted")
    COLAB_ENV = True
except ImportError:
    print("Running locally - skipping drive mount")
    COLAB_ENV = False

# Core imports for data handling and visualization
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model  # For architecture diagrams
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Setup complete. TensorFlow version:", tf.__version__)

Running locally - skipping drive mount
Setup complete. TensorFlow version: 2.19.0
